### 데이터 가져오기

하루의 binance api를 요청하여 5분 간격의 코인 시세 정보를 가져옵니다. 

In [1]:
import pandas as pd

from binance.client import Client # 바이낸스

In [2]:
# 바이낸스 5분봉
def get_5m_binance(start, end):

    # 회원가입 후 key를 발급받는다.
    api_key = '입력'
    api_secret = '입력'
    client = Client(api_key=api_key, api_secret=api_secret)
    
    minute = client.get_historical_klines(
        symbol='BTCUSDT',
        interval='5m',
        start_str=start,
        end_str=end
        # limit=200 # max = 1000
    )
    
    columns = ['Open time', 'Open', 'High', 'Low', 'Close',
               'Volume', 'Close time', 'Quote asset volume', 'Number of trades',
               'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
    
    binance = pd.DataFrame(minute, columns=columns)
    binance['Open time'] = pd.to_datetime(binance['Open time'], unit='ms')
    
    return binance

In [3]:
binance_2022 = get_5m_binance('2022-01-01', '2023-01-01') # 2022년
binance_2022 = binance_2022.drop(binance_2022.index[-1]) # 마지막 인덱스 제외('2023-01-01 00:00:00')
binance_2022.to_csv('DATA/Binance_BTC_prices_2022_5min.csv', index=False) # csv 파일로 저장

In [4]:
binance_2023 = get_5m_binance('2023-01-01', '2024-01-01') # 2023년
binance_2023 = binance_2023.drop(binance_2023.index[-1])
binance_2023.to_csv('DATA/Binance_BTC_prices_2023_5min.csv', index=False) # csv 파일로 저장

In [5]:
# 데이터 개수 검사
print(binance_2022.shape)
print(binance_2023.shape) # 2023년도 데이터가 모자르다.
print(365*288)

(105120, 12)
(105104, 12)
105120


### 2023년도 바이낸스 결측치 확인

In [6]:
binance_2023['Open time'] = pd.to_datetime(binance_2023['Open time'])

In [7]:
binance_2023['month'] = binance_2023['Open time'].dt.month
binance_2023['day'] = binance_2023['Open time'].dt.day
binance_2023['hour'] = binance_2023['Open time'].dt.hour

In [8]:
check = binance_2023.groupby(['month', 'day']).size()
check[check != 288] # 하루에 288개의 캔들이 나와야 한다.

month  day
3      24     272
dtype: int64

In [9]:
binance_3_24 = binance_2023[(binance_2023['month'] == 3) & (binance_2023['day'] == 24)]
binance_3_24.groupby('hour').size()

hour
0     12
1     12
2     12
3     12
4     12
5     12
6     12
7     12
8     12
9     12
10    12
11    12
12     8
14    12
15    12
16    12
17    12
18    12
19    12
20    12
21    12
22    12
23    12
dtype: int64

- 바이낸스 2023년도 데이터 부족한 이유 서칭
- https://www.binance.com/en/support/announcement/binance-spot-trading-system-temporary-maintenance-complete-2023-03-24-813a31506e9f478ea8c1058b425df87a
- binance 3월 24일 일시점검 14:00 거래 재개